In [1]:
# standarad imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import unicodedata

# import regular expression operations
import re

# import natural language toolkit
import nltk
# import stopwords list
from nltk.corpus import stopwords
# import wordcloud
from wordcloud import WordCloud

import requests
from bs4 import BeautifulSoup
import os
from pprint import pprint 
import time
import json

import acquire as a
import wrangle as w

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression



SyntaxError: invalid decimal literal (wrangle.py, line 162)

# acquire

In [ ]:
data = pd.read_json('github_data.json')
data

In [ ]:
df = pd.DataFrame(data)


# Prepare!

In [ ]:
#df = df.dropna()

In [ ]:
# text cleaning on a df column named 'readme'. removing special character and words.
df.readme_contents = df.readme_contents.str.replace('[/,_,-,:,"]', '', regex=True)

df.readme_contents = df.readme_contents.str.replace('heavy', '').str.replace('check', '').str.replace('mark', '')

In [ ]:
df.language.value_counts()


In [ ]:
df['clean_norm_token'] = df.readme_contents.apply(w.tokenize).apply(w.basic_clean).apply(w.remove_stopwords).apply(w.lemmatize)


# tokenizes, cleans, remove stopwprds, lemmatize the text in the readme_contents and 
#stores the processed text in a new column clean_norm_token

df.clean_norm_token = df.clean_norm_token.str.replace('124', '')

# it removes any occurences of the string '124' from the clean_norm_token column

In [ ]:
#Combine 'Jupyter Notebook' with 'Other' and create a new value 'Other'
df['language'] = df['language'].replace({'Jupyter Notebook': 'Other'})

In [ ]:
#in language column replace language with other if it is not in the top 3 languages
top_5 = df.language.value_counts().head(5).index.tolist()
print(top_5)
df.language = df.language.apply(lambda x: x if x in top_5 else 'other')

In [ ]:
# List of languages to combine into "Other"
languages_to_combine = ['Jupyter Notebook', 'MATLAB', 'TeX', 'C#', 'Matlab', 'Objective-C', 'Dart']
# Replace the specified languages with "Other"
df['language'] = df['language'].replace(languages_to_combine, 'other')
df

In [ ]:
df.language.value_counts()


In [ ]:
#baseline is 47%
df.language.value_counts(normalize=True)
#the relative frequency of each programming language in  dataset.

# explore

In [ ]:
#calls the split function from wrangle.py
train, validate, test = w.split_function_cat_target(df, "language")

In [ ]:
train.head()

In [ ]:
python_words = w.clean(' '.join(train[train.language=='Python']['clean_norm_token']))
jupyter_notebook_words = w.clean(' '.join(train[train.language=='Jupyter Notebook']['clean_norm_token']))
html_words = w.clean(' '.join(train[train.language=='HTML']['clean_norm_token']))
r_words = w.clean(' '.join(train[train.language=='R']['clean_norm_token']))
other_words = w.clean(' '.join(train[train.language=='other']['clean_norm_token']))

all_words = w.clean(' '.join(df['clean_norm_token']))
#various programming languages and aggregating the words associated with each language

In [ ]:
#how often each word appears in various lists of words, 
#possibly corresponding to different programming languages or technologies.
python_freq = pd.Series(python_words).value_counts()
jupyter_notebook_freq = pd.Series(jupyter_notebook_words).value_counts()
html_freq = pd.Series(html_words).value_counts()
r_freq = pd.Series(r_words).value_counts()
other_freq = pd.Series(other_words).value_counts()

all_freq = pd.Series(all_words).value_counts()

In [ ]:
def make_ngrams(words, n):
    return pd.Series(nltk.ngrams(words, n)).value_counts().head(20)
    

In [ ]:
def plot_bigrams(words):
    word_data = {k[0] + ' ' + k[1]: v for k, v in words.to_dict().items()}
    
    word_img = WordCloud(background_color='white', width=800, height=400).generate_from_frequencies(word_data)
    
    plt.figure(figsize=(8, 4))
    plt.imshow(word_img)
    plt.axis('off')
    plt.show()

In [ ]:
plot_bigrams(make_ngrams(python_words,2))


In [ ]:
plot_bigrams(make_ngrams(other_words,2))


In [ ]:
pd.Series(nltk.ngrams(python_words,2)).value_counts().head(20).plot.barh()
#horizontal bar chart of the 20 most frequent bigrams  in the list python_words

In [ ]:
pd.Series(nltk.ngrams(other_words,2)).value_counts().head(20)


# explore questions

# 1. What are most frequent used words in python/other breast cancer repos?

In [ ]:
word_counts[['python']].sort_values(by='python', ascending=False).head()


In [ ]:
blog_img = WordCloud(background_color='white').generate_from_frequencies(python_freq.head(50))
plt.figure(figsize=(8, 4))
plt.imshow(blog_img)
plt.axis('off')
plt.show()

In [ ]:
word_counts[['other']].sort_values(by='other', ascending=False).head()


In [ ]:
blog_img = WordCloud(background_color='white').generate_from_frequencies(other_freq.head(30))
plt.figure(figsize=(8, 4))
plt.imshow(blog_img)
plt.axis('off')
plt.show()